In [1]:
import numpy as np
import matplotlib.pyplot as plt

#import our custom utility class for activation functions
from neural_util import sigmoid, relu, sigmoid_backward, relu_backward

Functions we need to make: 

*   def initialize_parameters(layer_dims):

    return parameters

*   def model_forward(X, parameters):

    return AL, caches
    
*   def compute_cost(AL, Y):

    return cost
    
*   def model_backward(AL, Y, caches):

    return grads
    
*   def update_parameters(parameters, grads, learning_rate):
    
    return parameters



In [2]:
def initialize_parameters(layer_dims):
    """
    Arguments:
    layer_dims -- python array (list) containing the dimensions of each layer in our network
    
    Returns:
    parameters -- python dictionary containing your parameters "W1", "b1", ..., "WL", "bL":
                    Wl -- weight matrix of shape (layer_dims[l], layer_dims[l-1])
                    bl -- bias vector of shape (layer_dims[l], 1)
    """
    parameters = {}
    L = len(layer_dims)
    for l in range(1, L):
        # dimension of w is (l, l-1)
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l - 1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))
    return parameters


In [3]:
# simple forward function to compute z
def linear_forward(A, W, b):
    """
    Implement the linear part of a layer's forward propagation.

    Arguments:
    A -- activations from previous layer (or input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)

    Returns:
    Z -- the input of the activation function, also called pre-activation parameter 
    cache"""
    Z = np.dot(W, A) + b
    #in cache we'll save A, W, b
    cache = (A, W, b)

    return Z, cache

In [4]:
def linear_activation_forward(A_prev, W, b, activaton):
    """
    Implement the forward propagation for the LINEAR->ACTIVATION layer

    Arguments:
    A_prev -- activations from previous layer (or input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"

    Returns:
    A -- the output of the activation function, also called the post-activation value 
    cache -- a python tuple containing "linear_cache" and "activation_cache";
             stored for computing the backward pass efficiently
    """
    if activation == "sigmoid":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
    elif activation == "relu":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
        
    #linear_cache contains A_prev, W, b used for forward pass
    #activation_cache contains output of activation func
    cache = (linear_cache, activation_cache)

    return A, cache

In [6]:

def model_forward(X, parameters):
    """
    Implement forward propagation for the [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID computation
    
    Arguments:
    X -- data, numpy array of shape (input size, number of examples)
    parameters -- output of initialize_parameters_deep()
    
    Returns:
    AL -- activation value from the output (last) layer
    caches -- list of caches containing:
                every cache of linear_activation_forward() (there are L of them, indexed from 0 to L-1)
    """
    #caches contains linear & activation cache for all layers
    caches = [] 
    A = X
    L = len(layer_dims)
    #range iter below will forward pass w relu up to before the output layer
    for l in range(1, L)
        A_prev = A
        # use relu for forward pass
        A, cache = linear_activation_forward(A_prev, parameters['W' + str(l)], parameters['b' + str(l)], 'relu')
        #cache contains both linear_cache and activation_cache
        caches.append(cache)
    #compute forward pass for output layer
    AL, cache = linear_activation_forward(A, parameters['W' + str(L)], parameters['b' + str(L)], 'sigmoid')
    caches.append(cache)
    
    return AL, caches

In [10]:
# implement log loss function to create learning metric

def compute_cost(AL, Y):
    """
    Arguments:
    AL -- probability vector corresponding to your label predictions, shape (1, number of examples)
    Y -- true "label" vector (for example: containing 0 if non-cat, 1 if cat), shape (1, number of examples)

    Returns:
    cost -- cross-entropy cost
    """
    
    #get num of training examples
    m = Y.shape[1]
    cost = (-1/m) * np.sum(Y*np.log(AL) + (1-Y)*np.log(1-AL))
    
    # To make sure your cost's shape is what we expect (e.g. this turns [[17]] into 17)
    # np.squeeze ensures we have the correct shape, e.g instead of [[x]], we get X
    cost = np.squeeze(cost)
    
    return cost

In [11]:
def linear_backward(dZ, cache):
    #get params 
    A_prev, W, b = cache
    # get # examples
    m = A_prev.shape[1]
    
    dW = 1/m * (np.dot(dZ, A_prev.T))
    db = 1/m * (np.sum(dZ, axis = 1, keepdims = True))
    dA_prev = np.dot(cache[1].T, dZ) 
    
    return dA_prev, dW, db
    

In [12]:
def linear_backward_activation(dA, cache, activation):
    """
    Implement the backward propagation for the LINEAR->ACTIVATION layer.
    
    Arguments:
    dA -- post-activation gradient for current layer l 
    cache -- tuple of values (linear_cache, activation_cache) we store for computing backward propagation efficiently
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"
    
    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        # compute dZ given inputs
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, cache)
        
    elif activation == "sigmoid":
        dZ = sigmoid_backward(dZ, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, cache)
    
    return dA_prev, dW, db

In [14]:
def model_backward(AL, Y, caches):
    """
    Implement the backward propagation for the [LINEAR->RELU] * (L-1) -> LINEAR -> SIGMOID group
    
    Arguments:
    AL -- probability vector, output of the forward propagation (L_model_forward())
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat)
    caches -- list of caches containing:
                every cache of linear_activation_forward() with "relu" (it's caches[l], for l in range(L-1) i.e l = 0...L-2)
                the cache of linear_activation_forward() with "sigmoid" (it's caches[L-1])
    
    Returns:
    grads -- A dictionary with the gradients
             grads["dA" + str(l)] = ... 
             grads["dW" + str(l)] = ...
             grads["db" + str(l)] = ... 
    """
    #create dictionary to save gradients
    grads = {}
    #number of layers in network
    L = len(caches)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # make Y same shape as AL
    
    #initialize backprop
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    
    #performing backward pass for output layer using sigmoid_backward choice
    current_cache = caches[L-1]
    dA_prev_temp, dW_temp, db_temp = linear_backward_activation(AL, current_cache, 'sigmoid')
    
    #store gradients
    grads["dA" + str(L-1)] = dA_prev_temp
    grads["dW" + str(L)] = dW_temp
    grads["db" + str(L)] = db_temp 
    
    # loop in reverse from l = L-2 to l = 0
    for l in reversed(range(L-1)):
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(dA_prev_temp, caches[l], 'relu')
        grads["dA" + str(l-1)] = dA_prev_temp
        grads["dW" + str(l)] = dW_temp
        grads["db" + str(l)] = db_temp
    
    return grads

In [17]:
def update_parameters(params, grads, learning_rate):
    """
    Update parameters using gradient descent
    
    Arguments:
    params -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients, output of L_model_backward
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
                  parameters["W" + str(l)] = ... 
                  parameters["b" + str(l)] = ...
    """
    parameters = params.copy()
    # number of layers, remember params contains both W & b, so we divide by 2
    L = len(parameters) // 2
    
    #start updating!
    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W"+str(l+1)] - learning_rate * grads["dW"+str(l+1)]
        parameters["b" + str(l+1)] = parameters["b"+str(l+1)] - learning_rate * grads["db"+str(l+1)]
    
    return parameters